In [ ]:
! pip install qiskit

In [ ]:
! pip install quandl

In [ ]:
from qiskit import BasicAer
from qiskit.circuit.library import TwoLocal
# from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import WikipediaDataProvider, DataOnDemandProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import datetime
from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import LinearEqualityToPenalty
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing
from qiskit.optimization.converters import IntegerToBinary
from qiskit.optimization.converters import InequalityToEquality
import itertools
import quandl
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

quandl.ApiConfig.api_key = 'jw64yRHHgx8L-cYm_8_3'

In [ ]:
# Oracle is ORCL, Accenture is ACN, Hewlett Packard Enterprise is HPE, Cognizant is CTSH and Microsoft Corporation is MSFT

stocks = ["IBM","MSFT","ACN","ORCL","HPE","TCS","CTSH"]

# Extract free available stock price data from quandl, from 01.01.2016 to 31.12.2017

data = WikipediaDataProvider(token='jw64yRHHgx8L-cYm_8_3',tickers=stocks, start=datetime.datetime(2016,1,1), end=datetime.datetime(2017,12,31))
data.run()

# Calculate the means and covariates return
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

In [ ]:
for (cnt, s) in enumerate(data._tickers):
    plt.plot(data._data[cnt], label=s)
plt.legend()
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Set the parameters

q = 0.5 # set risk factor
num_assets = len(stocks)
budget = int(num_assets / 2) # set budget

### Building the QUBO representation of the problem

In [ ]:
mdl = Model("docplex model")
N = range(0,num_assets)
idz = [i for i in N]
z = mdl.binary_var_dict(idz)

# Add the constraint with the budget
mdl.add_constraint(mdl.sum(z[x] for x in range(0,num_assets)) == budget)

# Objective function to minimize 
mdl.minimize(mdl.sum(q*z[x]*2*sigma[x,y]*z[y] - mu[x]*z[x] for (x,y) in itertools.product(list(range(0,num_assets)), repeat=2)))

mod = QuadraticProgram()
mod.from_docplex(mdl)
print(mod.export_as_lp_string())

### Constraints to penality

In [ ]:
# Here the linear constraints are added to the penalty to have a Quadratic Unconstrained Binary Optimization problem. 

lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.encode(mod,penalty_factor=1e2)
print(qubo.export_as_lp_string())

In [ ]:
# We build the resulting Ising Hamiltonian from the QUBO above

qp2op = QuadraticProgramToIsing()
op, offset = qp2op.encode(qubo)
print('offset: {}'.format(offset))
print('operator:')
print(op.print_details())

### Solution with classical ML method

In [ ]:
exact_mes = NumPyMinimumEigensolver()
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
exact_result = exact.solve(qubo)
print(exact_result)

In [ ]:
ml_solution = [1,2,3] # the solution returned
stocks = np.array(stocks)
print("The assets to invest in, given the stock prices, the risk factor and the budget, are")
print(*stocks[ml_solution],sep=", ")

### Solution with QAOA

In [ ]:
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('qasm_simulator'))
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

In [ ]:
qaoa_solution = [1,2,3] # the solution returned
print("The assets to invest in, given the stock prices, the risk factor and the budget, are")
print(*stocks[qaoa_solution],sep=", ")

### Accuracy of the solution

In [ ]:
# Oracle is ORCL, Accenture is ACN, Hewlett Packard Enterprise is HPE, Cognizant is CTSH and Microsoft Corporation is MSFT

stocks = ["IBM","MSFT","ACN","ORCL","HPE","TCS","CTSH"]

# Extract free available stock price data from quandl, from 01.01.2018 to 31.03.2018

data_test = WikipediaDataProvider(token='jw64yRHHgx8L-cYm_8_3',tickers=stocks, start=datetime.datetime(2018,1,1), end=datetime.datetime(2018,3,31))
data_test.run()

In [ ]:
for (cnt, s) in enumerate(data_test._tickers):
    plt.plot(data_test._data[cnt], label=s)
plt.legend()
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Return the mean return per day

x = [0,1,1,1,0,0,0] # the solution returned
weights = data_test.get_mean_vector()
mu_test = data_test.get_period_return_mean_vector()

portfolio_return_per_day = np.sum(mu_test[x] * weights[x])
portfolio_return_per_day